In [38]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, get_scorer_names

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('../data/PC/train.csv',index_col=None)
df.drop(['assessed_2019','building_value_2019','land_value_2019'],axis=1,inplace=True)

In [5]:
X = df.drop(columns=['acct','TARGET'])
y = df['TARGET']

# Ridge CV

In [6]:
# Try a range of alpha values
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

In [32]:
# Ridge with cross-validation (default 5-fold)
ridge_cv = RidgeCV(alphas=alphas, scoring='neg_root_mean_squared_error')

In [33]:
ridge_cv.fit(X, y)

RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0, 100.0],
        scoring='neg_root_mean_squared_error')

In [34]:
# Best alpha
print(f"Best alpha (Ridge): {ridge_cv.alpha_}")

# RMSE on training data
y_pred = ridge_cv.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Train RMSE (Ridge): {rmse:.4f}")

Best alpha (Ridge): 0.01
Train RMSE (Ridge): 485246960543.1921


In [29]:
test = pd.read_csv('../data/PC/test.csv',index_col=None)

In [30]:
X_test = test.drop(columns=['acct'])  # or drop other non-feature cols
acct_ids = test['acct']               # store acct ID

In [ ]:
y_pred = ridge_cv.predict(X_test)

# LASSO ($\ell_1$)

In [37]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(alphas=[0.001, 0.01, 0.1, 1.0, 10], cv=5)
lasso_cv.fit(X, y)

print(f"Best alpha (Lasso): {lasso_cv.alpha_}")
print(f"Non-zero features: {(lasso_cv.coef_ != 0).sum()} / {len(lasso_cv.coef_)}")

y_pred = lasso_cv.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Train RMSE (Lasso): {rmse:.4f}")

C:\Users\sethk\anaconda3\envs\advent\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 459801273717013.25, tolerance: 5038755442665.565
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\sethk\anaconda3\envs\advent\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 451782590262519.56, tolerance: 5038755442665.565
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\sethk\anaconda3\envs\advent\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 445862003345523.25, tolerance: 5038755442665.565
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\sethk\anaconda3\envs\advent\lib\site-packages\

Best alpha (Lasso): 10.0
Non-zero features: 101 / 125
Train RMSE (Lasso): 43164.6058


In [40]:
X_test = test.drop(columns=['acct'])  # or drop other non-feature cols
acct_ids = test['acct']               # store acct ID

In [41]:
y_pred = lasso_cv.predict(X_test)

In [42]:
output_df = pd.DataFrame({
    'acct': acct_ids,
    'TARGET': y_pred
})

In [45]:
output_df.rename({'acct':'ACCOUNT'},axis=1,inplace=True)

In [46]:
output_df = output_df.drop_duplicates(subset='ACCOUNT')

In [47]:
output_df.reset_index(inplace=True,drop=True)

In [50]:
output_df.to_csv('prediction_3.csv',index=None)